# Introduction to blockchain HW 1

## Problem 1 (10 points)

Assume that at the end of the term an instructor uses an RSA Public Key Cryptosystem to sign the grades sent to students and to the Education Office. He signs each grade with his private key and transmits both the grade and the signature in a secure manner encrypted with a public key. After you decrypt the message you use his public key to decipher the signature and verify the grade. Assume the cryptosystem has the following features: n=55, e=33 and that the grading system features integer grades from 0 to 10. You have received the grade-signature pairs 8||13 and 6||41

1.Verify that the grades you received are indeed originating from the instructor showing just intermediate results of your calculation (2 points)

In [323]:
def res_decrypt(code):
    return pow(code, 33, 55)

In [324]:
print("13^33 mod 55 = ", res_decrypt(13))
print(res_decrypt(13) == 8)

13^33 mod 55 =  8
True


In [328]:
print("41^33 mod 55 = ", res_decrypt(41))
print(res_decrypt(41) == 6)

41^33 mod 55 =  6
True


2.Given the information above only can you fabricate the instructors’ signature for another grade? If yes, for which grade and how? (3 points)

In [329]:
i = 2
while pow(8,i,55) != 13:
    i += 1

In [330]:
pow(6, i, 55)

41

We fabricate insructor's key because he has chose too small numbers. Now I can fabricate all numbers.

Also, if we knew signatures for 2 and 3, we know signature for 6. It's problem of simple encription. If we encrypt, for example, a sum of certain grades, it will be harder to hack it.

3.What would you advise the instructor to do so as to prevent such fabrication? (2 points)

I advise to choose numbers that have more prime dividers. It will be harder to hack this.

4.Discuss what can go wrong in case the instructor sends a grade secretly by first enciphering it with a student’s public key, the signing it with his private key. Provide an example (3 points)

$m = D_s(E_p(m)) = E_p(D_s(m))$

In [331]:
pow(pow(8,33,55), 17,55)

8

## Problem 2 (10 points)

El-Gamal is videly used cryptographic standart. In this task you will implement El-Gamal encryption scheme using Python

1.Implement function for generating keys. The function must generate big random prime number (problem of generating big prime numbers was discussed during seminar after lecture 3). (2 points)

In [287]:
# Python implementation of Fermat's primality test to generate prime numbers of any bit length.

from random import randint

def is_prime(num, test_count):
    if num == 1:
        return False
    if test_count >= num:
        test_count = num - 1
    for x in range(test_count):
        val = randint(1, num - 1)
        if pow(val, num-1, num) != 1:
            return False
    return True

def generate_big_prime(n, test_count=1000):
    found_prime = False
    while not found_prime:
        p = randint(2**(n-1), 2**n)
        if is_prime(p, test_count):
            return p

2.Implement functions that realize the encryption and decryption functions. (2 points)

Key generation mechanism

1. Choose big prime number p
2. Choose two random numbers g and x 
3. Compute $y=g^x mod p$

Public key(y,g,p)

Secret key(x,g,p)

Encryption mechanism
1. Let message $M<p$
2. Choose random number k s.t.  1$<k<p−1$
3. Compute $a=g^k mod p$ and $b = y^k M mod p$ 

Ciphertext(a,b)

In [288]:
def encryption_mechanism(message, y, g, p):
    if (message >= p):
        raise Exception("Message must be < p")
    
    k = randint(1, p-1)
    a = pow(g, k, p)
    b = (pow(y, k) * message) % p
    return a, b

Decryption mechanism
$$M = \frac{b}{a^x}mod p$$

In [289]:
def decryption_mechanism(a, b, x, g, p):
    return (b * pow(a, p-1-x)) % p

3.Test your functions on random values and show that your realization works correctly (1 point)

In [290]:
n = 20
p = generate_big_prime(n) 
g = randint(1, 100)
x = randint(1, 100) # private key
y = pow(g, x, p) # public key
print("p=%i, g=%i, x=%i, y=%i" % (p, g, x, y))

message = randint(1, 1e5)
print("Initial message: %i" % message)

a, b =  encryption_mechanism(message, y, g, p)
print("Encrypted_message: a - %i, b - %i" % (a, b))

decrypted_message = decryption_mechanism(a, b, x, g, p)
print("Decrypted_message: %i" % decrypted_message)

p=843209, g=55, x=87, y=93442
Initial message: 69183
Encrypted_message: a - 637615, b - 828164
Decrypted_message: 69183


4.Implement functions that realize creation and verification of digital signature (2 points)

Sign(privatekey, m)

Choose arbitrary $k$ s.t. $(k, p − 1) = 1$

$a = g^k mod p$

$b = (m − xa)k^{−1} mod (p − 1)$

$ \sigma = (a, b)$

In [291]:
def gcd(a,b): 
    if(b == 0): 
        return a 
    else: 
        return gcd(b, a%b) 
    
def generate_less_than(p):
    found_gcd_1 = False
    while not found_gcd_1:
        k = randint(1,p-1)
        if gcd(k,p-1) == 1:
            return k

In [292]:
# Part of find_inverse below
# See: http://en.wikipedia.org/wiki/Extended_Euclidean_algorithm
def eea(a,b):
    if b==0:return (1,0)
    (q,r) = (a//b,a%b)
    (s,t) = eea(b,r)
    return (t, s-(q*t) )

# Find the multiplicative inverse of x (mod y)
# see: http://en.wikipedia.org/wiki/Modular_multiplicative_inverse
def find_inverse(x,y):
    inv = eea(x,y)[0]
    if inv < 1: inv += y #we only want positive values
    return inv

In [293]:
def sign(x, g, p, m):
    k = generate_less_than(p)
    a = pow(g, k, p)
    b = ((m - x*a) * find_inverse(k,p-1)) % (p-1)
    return (a, b)

Verify(publickey, m, $\sigma$): $y^a a^b mod p = g^m mod p$

In [294]:
def verify(a, b, y, g, p, m):
    return ((pow(y,a) * pow(a,b)) % p == pow(g,m,p))

5.Test your functions on random values and show that your algorithm works correctly (1 point)

In [295]:
(a, b) = sign(x, g, p, message)

In [296]:
verification = verify(a, b, y, g, p, message)

In [297]:
if (verification):
    print('All is right')
else:
    print('Something went wrong')

All is right


In [63]:
verification = verify(a-1, b, y, g, p, message)

In [64]:
if (verification):
    print('All is right')
else:
    print('Something went wrong')

Something went wrong


## Problem 3 (15 points)

1.Implement SHA256 (https://en.wikipedia.org/wiki/SHA-2) hashing algorithm using the pseudo-code below. Note the great increase in mixing between bits of the w[16..63] words compared to SHA-1 (10 points)

In [280]:
initial_hash_values=[
'6a09e667','bb67ae85','3c6ef372','a54ff53a',
'510e527f','9b05688c','1f83d9ab','5be0cd19'
]

sha_256_constants=[
'428a2f98','71374491','b5c0fbcf','e9b5dba5',
'3956c25b','59f111f1','923f82a4','ab1c5ed5',
'd807aa98','12835b01','243185be','550c7dc3',
'72be5d74','80deb1fe','9bdc06a7','c19bf174',
'e49b69c1','efbe4786','0fc19dc6','240ca1cc',
'2de92c6f','4a7484aa','5cb0a9dc','76f988da',
'983e5152','a831c66d','b00327c8','bf597fc7',
'c6e00bf3','d5a79147','06ca6351','14292967',
'27b70a85','2e1b2138','4d2c6dfc','53380d13',
'650a7354','766a0abb','81c2c92e','92722c85',
'a2bfe8a1','a81a664b','c24b8b70','c76c51a3',
'd192e819','d6990624','f40e3585','106aa070',
'19a4c116','1e376c08','2748774c','34b0bcb5',
'391c0cb3','4ed8aa4a','5b9cca4f','682e6ff3',
'748f82ee','78a5636f','84c87814','8cc70208',
'90befffa','a4506ceb','bef9a3f7','c67178f2'
]

def bin_return(dec):
    return(str(format(dec,'b')))

def bin_8bit(dec):
    return(str(format(dec,'08b')))

def bin_32bit(dec):
    return(str(format(dec,'032b')))

def bin_64bit(dec):
    return(str(format(dec,'064b')))

def hex_return(dec):
    return(str(format(dec,'x')))

def dec_return_bin(bin_string):
    return(int(bin_string,2))

def dec_return_hex(hex_string):
    return(int(hex_string,16))

def L_P(SET,n):
    to_return=[]
    j=0
    k=n
    while k<len(SET)+1:
        to_return.append(SET[j:k])
        j=k
        k+=n 
    return(to_return)

def s_l(bit_string):
    bit_list=[]
    for i in range(len(bit_string)):
        bit_list.append(bit_string[i])
    return(bit_list)

def l_s(bit_list):
    bit_string=''
    for i in range(len(bit_list)):
        bit_string+=bit_list[i]
    return(bit_string)

def rotate_right(bit_string,n):
    bit_list = s_l(bit_string)
    count=0
    while count <= n-1:
        list_main=list(bit_list)
        var_0=list_main.pop(-1)
        list_main=list([var_0]+list_main)
        bit_list=list(list_main)
        count+=1
    return(l_s(list_main))

def shift_right(bit_string,n):
    bit_list=s_l(bit_string)
    count=0
    while count <= n-1:
        bit_list.pop(-1)
        count+=1
    front_append=['0']*n
    return(l_s(front_append+bit_list))

def mod_32_addition(input_set):
    value=0
    for i in range(len(input_set)):
        value+=input_set[i]
    mod_32 = 4294967296
    return(value%mod_32)

def xor_2str(bit_string_1,bit_string_2):
    xor_list=[]
    for i in range(len(bit_string_1)):
        if bit_string_1[i]=='0' and bit_string_2[i]=='0':
            xor_list.append('0')
        if bit_string_1[i]=='1' and bit_string_2[i]=='1':
            xor_list.append('0')
        if bit_string_1[i]=='0' and bit_string_2[i]=='1':
            xor_list.append('1')
        if bit_string_1[i]=='1' and bit_string_2[i]=='0':
            xor_list.append('1')
    return(l_s(xor_list))

def and_2str(bit_string_1,bit_string_2):
    and_list=[]
    for i in range(len(bit_string_1)):
        if bit_string_1[i]=='1' and bit_string_2[i]=='1':
            and_list.append('1')
        else:
            and_list.append('0')

    return(l_s(and_list))

def or_2str(bit_string_1,bit_string_2):
    or_list=[]
    for i in range(len(bit_string_1)):
        if bit_string_1[i]=='0' and bit_string_2[i]=='0':
            or_list.append('0')
        else:
            or_list.append('1')
    return(l_s(or_list))

def not_str(bit_string):
    not_list=[]
    for i in range(len(bit_string)):
        if bit_string[i]=='0':
            not_list.append('1')
        else:
            not_list.append('0')
    return(l_s(not_list))

'''
SHA-256 Specific Functions:
'''

def Ch(x,y,z):
    return(xor_2str(and_2str(x,y),and_2str(not_str(x),z)))

def Maj(x,y,z):
    return(xor_2str(xor_2str(and_2str(x,y),and_2str(x,z)),and_2str(y,z)))

def e_0(x):
    return(xor_2str(xor_2str(rotate_right(x,2),rotate_right(x,13)),rotate_right(x,22)))

def e_1(x):
    return(xor_2str(xor_2str(rotate_right(x,6),rotate_right(x,11)),rotate_right(x,25)))

def s_0(x):
    return(xor_2str(xor_2str(rotate_right(x,7),rotate_right(x,18)),shift_right(x,3)))

def s_1(x):
    return(xor_2str(xor_2str(rotate_right(x,17),rotate_right(x,19)),shift_right(x,10)))

def message_pad(bit_list):
    pad_one = bit_list + '1'
    pad_len = len(pad_one)
    k=0
    while ((pad_len+k)-448)%512 != 0:
        k+=1
    back_append_0 = '0'*k
    back_append_1 = bin_64bit(len(bit_list))
    return(pad_one+back_append_0+back_append_1)

def message_bit_return(string_input):
    bit_list=[]
    for i in range(len(string_input)):
        bit_list.append(bin_8bit(ord(string_input[i])))
    return(l_s(bit_list))

def message_pre_pro(input_string):
    bit_main = message_bit_return(input_string)
    return(message_pad(bit_main))

def message_parsing(input_string):
    return(L_P(message_pre_pro(input_string),32))

def message_schedule(index,w_t):
    new_word = bin_32bit(mod_32_addition([int(s_1(w_t[index-2]),2),int(w_t[index-7],2),int(s_0(w_t[index-15]),2),int(w_t[index-16],2)]))
    return(new_word)

'''
This example of SHA_256 works for an input string <56 characters.
'''

def sha_256(input_string):
    w_t=message_parsing(input_string)
    a=bin_32bit(dec_return_hex(initial_hash_values[0]))
    b=bin_32bit(dec_return_hex(initial_hash_values[1]))
    c=bin_32bit(dec_return_hex(initial_hash_values[2]))
    d=bin_32bit(dec_return_hex(initial_hash_values[3]))
    e=bin_32bit(dec_return_hex(initial_hash_values[4]))
    f=bin_32bit(dec_return_hex(initial_hash_values[5]))
    g=bin_32bit(dec_return_hex(initial_hash_values[6]))
    h=bin_32bit(dec_return_hex(initial_hash_values[7]))
    for i in range(0,64):
        if i <= 15: 
            t_1=mod_32_addition([int(h,2),int(e_1(e),2),int(Ch(e,f,g),2),int(sha_256_constants[i],16),int(w_t[i],2)])
            t_2=mod_32_addition([int(e_0(a),2),int(Maj(a,b,c),2)])
            h=g
            g=f
            f=e
            e=mod_32_addition([int(d,2),t_1])
            d=c
            c=b
            b=a 
            a=mod_32_addition([t_1,t_2])
            a=bin_32bit(a)
            e=bin_32bit(e)
        if i > 15:
            w_t.append(message_schedule(i,w_t))
            t_1=mod_32_addition([int(h,2),int(e_1(e),2),int(Ch(e,f,g),2),int(sha_256_constants[i],16),int(w_t[i],2)])
            t_2=mod_32_addition([int(e_0(a),2),int(Maj(a,b,c),2)])
            h=g
            g=f
            f=e
            e=mod_32_addition([int(d,2),t_1])
            d=c
            c=b
            b=a 
            a=mod_32_addition([t_1,t_2])
            a=bin_32bit(a)
            e=bin_32bit(e)
    hash_0 = mod_32_addition([dec_return_hex(initial_hash_values[0]),int(a,2)])
    hash_1 = mod_32_addition([dec_return_hex(initial_hash_values[1]),int(b,2)])
    hash_2 = mod_32_addition([dec_return_hex(initial_hash_values[2]),int(c,2)])
    hash_3 = mod_32_addition([dec_return_hex(initial_hash_values[3]),int(d,2)])
    hash_4 = mod_32_addition([dec_return_hex(initial_hash_values[4]),int(e,2)])
    hash_5 = mod_32_addition([dec_return_hex(initial_hash_values[5]),int(f,2)])
    hash_6 = mod_32_addition([dec_return_hex(initial_hash_values[6]),int(g,2)])
    hash_7 = mod_32_addition([dec_return_hex(initial_hash_values[7]),int(h,2)])
    final_hash = (hex_return(hash_0),
                  hex_return(hash_1),
                  hex_return(hash_2),
                  hex_return(hash_3),
                  hex_return(hash_4),
                  hex_return(hash_5),
                  hex_return(hash_6),
                  hex_return(hash_7))
    return(final_hash) 

2.Calculate hashes of the texts below (1 point)

In [149]:
string_small = 'This is a very small string with a few characters.'
string_larger = 'This is a larger string that contains more characters.'
string_big = 'This is a larger string that contains more characters. This demonstrates that no matter how big the input stream is, the generated hash is the same size (but of course, not the same value). If two files have a different hash, they surely contain different data.'
string_empty = ''

In [151]:
print(sha_256(string_small))
print(sha_256(string_larger))
print(sha_256(string_big))
print(sha_256(string_empty))

('551bfc4b', '109bec23', 'bbf29ba0', 'e8c4520f', '194ae77d', 'c1839b0', '52552deb', '23774f07')
('e3c7ba4a', '5ff68765', '249cc065', 'a5198b1b', '8be94b67', '910a946f', '1fbaa995', 'daecb51a')
('4ce5fcc6', '71b42410', '7676b353', '776ec461', '5eb14897', '600d5022', 'b322738b', 'da198b16')
('e3b0c442', '98fc1c14', '9afbf4c8', '996fb924', '27ae41e4', '649b934c', 'a495991b', '7852b855')


In [314]:
def encrypt_string(hash_string):
    sha_signature = \
        hashlib.sha256(hash_string.encode()).hexdigest()
    return sha_signature

hash_string = string_small
sha_signature = encrypt_string(hash_string)
print(sha_signature)

551bfc4b109bec23bbf29ba0e8c4520f194ae77d0c1839b052552deb23774f07


3.What is a bit length of each hash? (1 point)

In [177]:
a = ''
for word in sha_256(string_larger):
    a += word
print(len(a))

64


256 bits long

Each digit (hexadecimal representation) codes for 4 bits

64 digits to represent 256 bits

4.What is the bitwise distance between them? What is bitwise distance between their hashes? (1 point)

In [164]:
def hamming2(s1, s2):
    """Calculate the Hamming distance between two bit strings"""
    assert len(s1) == len(s2)
    return sum(c1 != c2 for c1, c2 in zip(s1, s2))

In [165]:
hamming2('551bfc4b', '109bec23')

6

5.Typically use apply hash function to our passwords and texts that we want to digitally sign. Implement digital signature of hashed string using El-Gamal digital signature. Compare the digital signature of plain text and hashed text. (2 points)

In [319]:
str(message)

'69183'

In [316]:
res = sha_256(str(message))
res_str = ''.join(['%.2x' % int(i, 16) for i in res])
res_str

'ad3465b68e2299613bb846fa71fef2f0c818dbc47078f811f0596f14a9218fd0'

In [317]:
hash_string = str(message)
sha_signature = encrypt_string(hash_string)
print(sha_signature)

ad3465b68e2299613bb846fa71fef2f0c818dbc47078f811f0596f14a9218fd0


In [322]:
int(res_str, 16)

78342700850333459119618701488595620606496640477638269326528475266771455152080

In [321]:
verify(a, b, y, g, p, message)

True

In [318]:
verify(a, b, y, g, p, int(res_str, 16))

False

## Problem 4 (15 points)

Merkle hash trees play an important role in forming transaction blocks in blockchain. In this assignment we ask you to plot your own Merkle hash tree and check its' properties. Below we provide you with some code fragment what you can use in your assignment

In [7]:
import matplotlib
import networkx as nx
%matplotlib qt5
from networkx import balanced_tree, draw_networkx, draw

Let us plot graph basis for Merkle hash tree

In [279]:
G = nx.Graph()
positions = {}
coordinates = [
    [0, 4],
    [-2, 3],
    [2, 3],
    [-3, 2],
    [-1, 2],
    [1, 2],
    [3, 2],
    [-3, 1],
    [-1, 1],
    [1, 1],
    [3, 1]
]
parents = [0, 0, 0, 1, 1, 2, 2, 3, 4, 5, 6]
for index in range(11):
    G.add_node(index)
    G.add_edge(index, parents[index])
    positions[index] = coordinates[index]
nx.draw(G, positions, node_size = 1000)
labels = {
    0: '0',
    1: '1',
    2: '2',
    3: '3',
    4: '4',
    5: '5',
    6: '6',
    7: 'tx1',
    8: 'tx2',
    9: 'tx3',
    10: 'tx4',
}
nx.draw_networkx_labels(G, positions, labels = labels)

{0: Text(0,4,'0'),
 1: Text(-2,3,'1'),
 2: Text(2,3,'2'),
 3: Text(-3,2,'3'),
 4: Text(-1,2,'4'),
 5: Text(1,2,'5'),
 6: Text(3,2,'6'),
 7: Text(-3,1,'tx1'),
 8: Text(-1,1,'tx2'),
 9: Text(1,1,'tx3'),
 10: Text(3,1,'tx4')}

In Bitcoin double sha256 hash scheme is used. Here is an example.

In [118]:
import hashlib


first_hash = hashlib.sha256(b"hello") # "b" stands for binary representation
second_hash = hashlib.sha256()
print('First hash represented as a hexadecimal number:', first_hash.hexdigest())
second_hash.update(first_hash.digest())
print('Second hash represented as a hexadecimal number:', second_hash.hexdigest())

First hash represented as a hexadecimal number: 2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824
Second hash represented as a hexadecimal number: 9595c9df90075148eb06860365df33584b75bff782a510c6cd4883a419833d50


In [173]:
len('2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824')

64

Now we can easily change vertices' labels to hashes of corresponding messages and plot new graph

In [37]:
labels[3] = hashlib.sha256(hashlib.sha256(b"tx1").digest()).hexdigest()

# and plot the graph again

nx.draw(G, positions, node_size = 1000)
nx.draw_networkx_labels(G, positions, labels = labels, font_size = 8)
print(labels[3])

856a4921cd32690244af7568e7bd1391a94119e17c7f33234f4bf11271b223e5


1.Construct Merkle hash tree using previously constructed graph by finding corresponding SHA256 hashes on vertices (2 points). Plot obtained Merkle hash tree (1 point)

In [259]:
def double_hash(string):
    return hashlib.sha256(hashlib.sha256(string.encode('utf-8')).digest()).hexdigest()

In [268]:
def sum_hash(*args):
    return double_hash(''.join(args))

In [269]:
#labels[4] = hashlib.sha256(hashlib.sha256(b"tx2").digest()).hexdigest()
def plot_graph(myList = [], *args):
    labels[3] = double_hash(labels[7])
    labels[4] = double_hash(labels[8])
    labels[5] = double_hash(labels[9])
    labels[6] = double_hash(labels[10])
    labels[1] = sum_hash(labels[3],labels[4])
    labels[2] = sum_hash(labels[5],labels[6])
    labels[0] = sum_hash(labels[1],labels[2])

    nx.draw(G, positions, node_size = 1000)
    nx.draw_networkx_labels(G, positions, labels = labels, font_size = 8)
    return
plot_graph(labels)

2.Provide a proof of correctness of leaf tx2 (2 points). 

In [270]:
def hash_verification(my_hash,state):
    if my_hash==state:
        verification = True 
    else:
        verification = False
    
    return verification
    

In [271]:
temp2 = hashlib.sha256(b"tx2").digest()
print(temp2)
state1 = hashlib.sha256(b"'\xcad\xc0\x92\xa9Y\xc7\xed\xc5%\xedE\xe8E\xb1\xdeju\x90\xd1s\xfd/\xad\x913\xc8\xa7y\xa1\xe3").hexdigest()
print(state1)
print(labels[4])
print(hash_verification(labels[4], state1))

b"'\xcad\xc0\x92\xa9Y\xc7\xed\xc5%\xedE\xe8E\xb1\xdeju\x90\xd1s\xfd/\xad\x913\xc8\xa7y\xa1\xe3"
79043a4d1d4d6d0b830519bfc07b92b4d162a4cd54235719c2c3cc211a638dfd
79043a4d1d4d6d0b830519bfc07b92b4d162a4cd54235719c2c3cc211a638dfd
True


3.Provide a proof of correctness for set of leafs (tx3-tx4) (2 points)

In [272]:
temp3 = hashlib.sha256(hashlib.sha256(b"tx3").digest()).hexdigest()
temp4 = hashlib.sha256(hashlib.sha256(b"tx4").digest()).hexdigest()
temp34 = temp3 + temp4
print(temp34)
state2 = hashlib.sha256(hashlib.sha256(b'ef729c31d206229249bd791b29676d26cc7465aa6bc2003d80c7a82a316e02334746dc9c16f97469fa45710394c4a0e2f29226efc04cab47c29ce579ae19a74e').digest()).hexdigest()
print(state2)
print(labels[2])
print(hash_verification(labels[2], state2))

ef729c31d206229249bd791b29676d26cc7465aa6bc2003d80c7a82a316e02334746dc9c16f97469fa45710394c4a0e2f29226efc04cab47c29ce579ae19a74e
5c72c2fc6c2ed732c66fa8dc63f163617be1eb6175b88b3b640f440abb236e73
5c72c2fc6c2ed732c66fa8dc63f163617be1eb6175b88b3b640f440abb236e73
True


4.Change the value on leaf tx1 and recompute corresponding hashes. Plot newly obtained Merkle hash tree (2 points)

In [273]:
labels[7] = "tx0"
plot_graph(labels)

5.Nodes in Merkle hash trees may have arbitrary fanout. In previouse items we consider the case of fanout equals to two. But what will change if we set a fanout equals to three? Construct Merkle hash trees with fanout 3 to sign 9 values? Construct the hash tree with fanout 2 to sign the same set of values? Plot obtained trees (4 points) 

In [274]:
G1 = nx.Graph()
positions1 = {}
coordinates1 = [
    [0, 9],
    [-2, 8],
    [0, 8],
    [2, 8],
    [-4, 7],
    [-3, 7],
    [-2, 7],
    [-1, 7],
    [0, 7],
    [1, 7],
    [2, 7],
    [3, 7],
    [4, 7],
     [-4, 6],
    [-3, 6],
    [-2, 6],
    [-1, 6],
    [0, 6],
    [1, 6],
    [2, 6],
    [3, 6],
    [4, 6],
]
parents1 = [0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
for index in range(len(parents1)):
    G1.add_node(index)
    G1.add_edge(index, parents1[index])
    positions1[index] = coordinates1[index]
nx.draw(G1, positions1, node_size = 1000)
labels1 = {
    0: '0',
    1: '1',
    2: '2',
    3: '3',
    4: '4',
    5: '5',
    6: '6',
    7: '7',
    8: '8',
    9: '9',
    10: '10',
    11: '11',
    12: '12', 
    13: 'tx1',
    14: 'tx2',
    15: 'tx3',
    16: 'tx4',
    17: 'tx5',
    18: 'tx6',
    19: 'tx7',
    20: 'tx8',
    21: 'tx9',
}
nx.draw_networkx_labels(G1, positions1, labels = labels1)

{0: Text(0,9,'0'),
 1: Text(-2,8,'1'),
 2: Text(0,8,'2'),
 3: Text(2,8,'3'),
 4: Text(-4,7,'4'),
 5: Text(-3,7,'5'),
 6: Text(-2,7,'6'),
 7: Text(-1,7,'7'),
 8: Text(0,7,'8'),
 9: Text(1,7,'9'),
 10: Text(2,7,'10'),
 11: Text(3,7,'11'),
 12: Text(4,7,'12'),
 13: Text(-4,6,'tx1'),
 14: Text(-3,6,'tx2'),
 15: Text(-2,6,'tx3'),
 16: Text(-1,6,'tx4'),
 17: Text(0,6,'tx5'),
 18: Text(1,6,'tx6'),
 19: Text(2,6,'tx7'),
 20: Text(3,6,'tx8'),
 21: Text(4,6,'tx9')}

In [275]:
G = nx.Graph()
positions = {}
coordinates = [
    [0, 5],
    [-3.5, 4],
    [3.5, 4],
    [-5, 3],
    [-2, 3],
    [2, 3],
    [5, 3],
    [-6, 2],
    [-4, 2],
    [-3, 2],
    [-1, 2],
    [1, 2],
    [3, 2],
    [4, 2],
    [6, 2],
    [-7, 1],
    [-5, 1],
    [-4, 1],
    [-3, 1],
    [-1, 1],
    [1, 1],
    [3, 1],
    [4, 1],
    [6, 1],
    [-7, 0],
    [-5, 0]
]
parents = [0, 0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
for index in range(len(parents)):
    G.add_node(index)
    G.add_edge(index, parents[index])
    positions[index] = coordinates[index]
nx.draw(G, positions, node_size = 1000)
labels = {
    0: '0',
    1: '1',
    2: '2',
    3: '3',
    4: '4',
    5: '5',
    6: '6',
    7: '7',
    8: '8',
    9: '9',
    10: '10',
    11: '11',
    12: '12',
    13: '13',
    14: '14',
    15: '15',
    16: '16',
    17: 'tx1',
    18: 'tx2',
    19: 'tx3',
    20: 'tx4',
    21: 'tx5',
    22: 'tx6',
    23: 'tx7',
    24: 'tx8',
    25: 'tx9'
}
nx.draw_networkx_labels(G, positions, labels = labels)

{0: Text(0,5,'0'),
 1: Text(-3.5,4,'1'),
 2: Text(3.5,4,'2'),
 3: Text(-5,3,'3'),
 4: Text(-2,3,'4'),
 5: Text(2,3,'5'),
 6: Text(5,3,'6'),
 7: Text(-6,2,'7'),
 8: Text(-4,2,'8'),
 9: Text(-3,2,'9'),
 10: Text(-1,2,'10'),
 11: Text(1,2,'11'),
 12: Text(3,2,'12'),
 13: Text(4,2,'13'),
 14: Text(6,2,'14'),
 15: Text(-7,1,'15'),
 16: Text(-5,1,'16'),
 17: Text(-4,1,'tx1'),
 18: Text(-3,1,'tx2'),
 19: Text(-1,1,'tx3'),
 20: Text(1,1,'tx4'),
 21: Text(3,1,'tx5'),
 22: Text(4,1,'tx6'),
 23: Text(6,1,'tx7'),
 24: Text(-7,0,'tx8'),
 25: Text(-5,0,'tx9')}

In [276]:
for i in range(4, 13):
    labels1[i] = double_hash(labels1[i+9])
for i in range(1, 4):
    labels1[i] = sum_hash(labels1[3*i + 1],labels1[3*i + 2],labels1[3*i + 3])
labels1[0] = sum_hash(labels1[1],labels1[2],labels1[3])
nx.draw(G1, positions1, node_size = 1000)
nx.draw_networkx_labels(G1, positions1, labels = labels1, font_size = 8)

{0: Text(0,9,'e7296a950ce7691ac317379ed5ab311cb1cdd4904b59ed20d410bc51b7f69196'),
 1: Text(-2,8,'66ed30b63243fcd5bd1b7414b59e2c2346ea01ca124d58651653ee4e5f47a43b'),
 2: Text(0,8,'c8cfb3fc3818c0aa840299e590ce437472935b00cf9f2b49ea559f7e3f8c012f'),
 3: Text(2,8,'d9dccb4196250a8861ce62b1f8e7e44563c936562337a98cb39d5f69c4696501'),
 4: Text(-4,7,'856a4921cd32690244af7568e7bd1391a94119e17c7f33234f4bf11271b223e5'),
 5: Text(-3,7,'79043a4d1d4d6d0b830519bfc07b92b4d162a4cd54235719c2c3cc211a638dfd'),
 6: Text(-2,7,'ef729c31d206229249bd791b29676d26cc7465aa6bc2003d80c7a82a316e0233'),
 7: Text(-1,7,'4746dc9c16f97469fa45710394c4a0e2f29226efc04cab47c29ce579ae19a74e'),
 8: Text(0,7,'929e74a52e6f0d8390d60d631d6dc8fb6cde5f10c04c7053bc94ce3f62759823'),
 9: Text(1,7,'ccd702558bb588ba5d49d2ec85af0453fae434476f205ad6caa93da525f86627'),
 10: Text(2,7,'038edc078728e6455aa202890dec7d4635f2180e999f1a7c280c0283e5dc9ccf'),
 11: Text(3,7,'9cb58fa7c7cf21f3e426c113f8f48c115402a476024f21edd7db637f06bb8f76'),
 12: Text

In [278]:
for i in range(8, 17):
    labels[i] = double_hash(labels[i+9])
for i in range(7, -1, -1):
    labels[i] = sum_hash(labels[i*2 +1], labels[i*2 + 2])
nx.draw(G, positions, node_size = 1000)
nx.draw_networkx_labels(G, positions, labels = labels, font_size = 8)

{0: Text(0,5,'3a4b8b844d4877130956c6d81805eb2619bf15e56c2cc6f9044abbd49e681edf'),
 1: Text(-3.5,4,'6af4f6dc4bc0b1e5f302e90de9f0eb00d77c40667b4fc9c91c6a2029a31971d5'),
 2: Text(3.5,4,'762dea47265b09f5622bb4f683cacac619dd054acf790af83ecc38deca926819'),
 3: Text(-5,3,'7c61d61a4d6ed41357bec48e23bfe0b11466b7583c906eb1f9ae547edf9800dc'),
 4: Text(-2,3,'26d31590c0c2130ecc9e15a2f593d2d293b9d62674d5e060615506e204ed8ab9'),
 5: Text(2,3,'534280e78185f3d7097a6fc216fcfb2a1000c682e9e65d23d88c40b7e87c2a5e'),
 6: Text(5,3,'360013549010b5e10edfcd86290c6bb6e3020730c8b51d7c51b3fbeafc9796a5'),
 7: Text(-6,2,'f201e24fbdc25f61d0482caf9ec1107190f71f36223d96e55dd3cf2c6eba1cfc'),
 8: Text(-4,2,'856a4921cd32690244af7568e7bd1391a94119e17c7f33234f4bf11271b223e5'),
 9: Text(-3,2,'79043a4d1d4d6d0b830519bfc07b92b4d162a4cd54235719c2c3cc211a638dfd'),
 10: Text(-1,2,'ef729c31d206229249bd791b29676d26cc7465aa6bc2003d80c7a82a316e0233'),
 11: Text(1,2,'4746dc9c16f97469fa45710394c4a0e2f29226efc04cab47c29ce579ae19a74e'),
 12

6.What is the optimum tree fanout for signing first 4 values in the set up of previouse item? (2 points)

When we use fanout=2, we need $2*n$ memory for hashes

When, for example, we use fanout=4, we need $1.33*n$ memory, but to calculate every node we use more cpu.

For 4 values, probably, we could use fanout=4, not to do extra calculations of the sum, but string for hashing wil be bigger, and more computationally difficult.